In [7]:
import numpy as np
import torch
from torch import nn
import os
from tqdm.notebook import tqdm

In [8]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, self.hidden_size).to(x.device) 
        c0 = torch.zeros(self.num_layers * 2, self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out


In [9]:
# Prepare data
X = []
y = []
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X.append(file['features'])
        y.append(file['labels'])
N = len(y)

X_test = X[0:N//4]
y_test = y[0:N//4]

X_train = X[N//4:N]
y_train = y[N//4:N]
n = len(y_train)
# print(torch.cuda.is_available())

In [10]:

def train(inputs, outputs):
    # Parameters
    hidden_size = 256
    input_size = 105
    output_size = 7
    num_layers = 2
    num_epochs = 100
    learning_rate = 0.01
    batch_size = 32

    # Instantiate the model
    model = LSTMModel(input_size, hidden_size, num_layers, output_size)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device='cuda')
    criterion = criterion.to(device='cuda')

    n = len(inputs)
    # Training loop
    for epoch in range(num_epochs):
        total_loss=0
        slices = np.random.randint(0,n-1,(batch_size,))
        for j in tqdm(range(batch_size)):
            i = slices[j]
            input = torch.tensor(inputs[i], dtype=torch.float32).to(device='cuda')
            output = torch.tensor(outputs[i], dtype=torch.long).to(device='cuda')
            output_pred = model(input)
            optimizer.zero_grad()
            loss = criterion(output_pred, output)
            total_loss += loss
            loss.backward()
            optimizer.step()

        # if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item()}')
    return model


In [11]:
model = train(X_train, y_train)

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1/100, Loss: 25.896987915039062


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 2/100, Loss: 23.696809768676758


  0%|          | 0/32 [00:00<?, ?it/s]

...

Epoch 98/100, Loss: 7.92908239364624


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 99/100, Loss: 8.804515838623047


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 100/100, Loss: 8.215948104858398


In [23]:
import scoring

def predict(model, inputs):
    y = []
    for input in tqdm(inputs):
        y += predict_one(model, input)
        # print(y)
    return y

def predict_one(model, input):
    output_pred = model(torch.tensor(input, dtype=torch.float32).to(device='cuda'))
    N = len(input)
    y = []
    for i in range(N):
        mx = output_pred[i][0]
        p = 0
        for j in range(1,output_pred.shape[1]):
            if output_pred[i][j] > mx:
                mx = output_pred[i][j]
                p = j
        y.append(p)
    return y


y_pred = predict(model, X_train)


  0%|          | 0/115 [00:00<?, ?it/s]

train score:


TypeError: only length-1 arrays can be converted to Python scalars

In [30]:
print('train score:')
y_true = []
for y in y_train:
    y_true += y.tolist()
print(len(y_pred))
print(len(y_true))
scoring.score(y_pred, y_true)


train score:
293884
293884
Class 0: Precision: 0.959, Recall: 0.988
Class 1: Precision: 0.519, Recall: 0.334
Class 2: Precision: 0.821, Recall: 0.857
Class 3: Precision: 0.628, Recall: 0.131
Class 4: Precision: 0.653, Recall: 0.440
Class 5: Precision: 0.825, Recall: 0.797
Class 6: Precision: 0.890, Recall: 0.565
mf1 score: 0.661
acc_score: 0.913
kappa_score: 0.983


c:\Users\29485\Desktop\shtech\cs182\project\iml-project\scoring.py:46: RuntimeWarning: overflow encountered in scalar add
  pe += np.dot(cfm[:,i], cfm[i,:])


  0%|          | 0/38 [00:00<?, ?it/s]

test score:


ValueError: operands could not be broadcast together with shapes (0,) (2497,) 

In [ ]:
y_pred = predict(model, X_test)


In [31]:
print('test score:')
y_true = []
for y in y_test:
    y_true += y.tolist()
scoring.score(y_pred, y_true)

test score:
Class 0: Precision: 0.907, Recall: 0.987
Class 1: Precision: 0.134, Recall: 0.153
Class 2: Precision: 0.702, Recall: 0.687
Class 3: Precision: 0.255, Recall: 0.018
Class 4: Precision: 0.225, Recall: 0.024
Class 5: Precision: 0.694, Recall: 0.529
Class 6: Precision: 0.000, Recall: 0.000
mf1 score: nan
acc_score: 0.846
kappa_score: 0.697


c:\Users\29485\Desktop\shtech\cs182\project\iml-project\scoring.py:27: RuntimeWarning: invalid value encountered in scalar divide
  pre += TP / (TP + FP)
